<a href="https://colab.research.google.com/github/huongd17at089/rice_diseases_detection/blob/master/rice_diseases_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import cv2

path = ""
label_dic = {
    "Healthy" : 0,
    "BrownSpot": 1,
    "Hispa": 2,
    "LeafBlast" : 3

}

In [3]:
def get_data(path, img_size):
  data = []
  for root, dirs, files in os.walk(path):
    label = label_dic.get(list(root.split(sep="/"))[-1], None)
    if label :
      for f in files:
        try: 
          img_path = os.path.join(root,f)
          img = cv2.imread(img_path) #cv2.IMREAD_GRAYSCALE
          img = cv2.resize(img,(img_size,img_size))
          data.append[img, label]
        except Exception as e:
          print(e)
  return np.array(data)